In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import requests
import time
import datetime
import numpy as np
import datetime

In [ ]:
# opciones de navegación
# ahora si se puede laburar
opt =  webdriver.ChromeOptions()
opt.add_argument('--start-maximized')
opt.add_argument('--disable-extensions')


driver_path = r'/home/frangr94/Desktop/proyecto_scrapper/webdriver/chromedriver.exe'
driver = webdriver.Chrome(options=opt)

# inicializar navegador y cargar página completa, espefificar día y hora
date_time = str(datetime.datetime.now())
megatonetienda='https://www.megatone.net/search/{}/'
articulo=['auriculares','notebooks','lavado']



In [ ]:
# ir a la busqueda
# probemos si es mejor asi
data=[]
for art in articulo:
    buscar=megatonetienda.format(art)
    driver.get(buscar)
    time.sleep(10)
    print("se busca en",buscar,"en fecha y hora:",date_time)


    # scrapear data scaneando todas las páginas (especificar rango)
    itemlist = []
    for i in range (100):
        page=i+1
        items = driver.find_elements(By.CSS_SELECTOR,'div.w100.dIB.AnchoInterno.TextDeco.pR')
        for i in range(len(items)):
            itemlist.append(items[i].text)
        itemlist = list(filter(None,itemlist))
        arrayitems=[]
        for i in range (len(itemlist)):
       
            words=str(itemlist[i]).split('\n')
            words=[x for x in words if 'Sin Interés' not in x]
            words=[x for x in words if 'OFF' not in x]
            words=[x for x in words if 'Envío' not in x]
            words=[x for x in words if 'GRATIS' not in x]
            if len(words)<3:
                words.append('null')
            words.append(date_time)
            words.append(art)
            print(words)

            data.append(words)
            
        

        #driver.execute_script("window.scrollBy(0,3500)")
        try:
            nextpage = driver.find_element(By.XPATH, '//*[@id="botones"]/button[{}]'.format(page))
            #actions= ActionChains(driver)
            #actions.move_to_element(nextpage).perform
            driver.execute_script("window.scrollBy(0,2300)")
            time.sleep(2)
            nextpage.click()
            time.sleep(2)
        
        except:
            print('ya esta')
            break
    


In [ ]:
print(len(data))

In [ ]:
# crear csv y etiquetar con fecha actual
df = pd.DataFrame(list())
df.to_csv('data.csv')

import os
if os.path.exists('data.csv'):
    os.remove('data.csv')
else:
    print('creando data.csv')

import csv
fields = ['producto','precio','precio_rebaja','date_time','categoria']
rows = data

with open ('data.csv', 'w', newline='') as f:
    write = csv.writer(f, delimiter='#')
    write.writerow(fields)
    write.writerows(rows)

In [ ]:
#esto es un cambio